# Optical Character Recognition(Optimisation)
 
**Contents :**
	1. Image Source
	2. Deskewing
	3. Character Recognising Algorithm
		- Clustering of points in an Image(Land marks)
		- Representing Images
        - Finding Shortest Path
        - Working with Dictionaries and Pickle files

In [1]:
import numpy as np
import matplotlib.pyplot as graph
import scipy as sp
import math
import mnist
from sklearn.cluster import KMeans
import matrix2 as matrix2

train_images= mnist.train_images()
train_labels= mnist.train_labels()
test_images= mnist.test_images()
test_labels= mnist.test_labels()

In [2]:
###########       EUCIDIAN-DISTANCES     #############
import matrix2 as matrix2  # This is file imported consists of shortest path algorithms

def euclid(x1,y1,x2,y2):   # This function calculates the Euclidian distance between two points
    d= np.sqrt((y2-y1)**2+(x2-x1)**2)
    return(d)

def fun(lm,i,n):  # Finds distance to every point to the other point
    x1,y1,x2,y2= 0,0,0,0
    lm1=[]
    for j in range(0,n):
        x1,y1= lm[i,0],lm[i,1]
        x2,y2= lm[j,0],lm[j,1]
        lm1.append(euclid(x1,y1,x2,y2))
    return(lm1)

def fun1(lm,n):  # All that lists of distances will be inserted in a single list.
    lm2= []
    for i in range(0,n):
        lm2.append(fun(lm,i,n))
    return(lm2)


# Deskewing

When we write, we often write at angles to the paper, which cause letters and numbers to be skewed. Unfortunately, unlike the human eye, computers cannot easily find similarities between images that are transformations of each other. Thus, the process of deskewing

Very formally, deskewing is the process of straightening an image that has been scanned or written crookedly — that is an image that is slanting too far in one direction, or one that is misaligned.

In [3]:
###########     DESKEWING     ###########
# This is the code which is taken from the internet.

from scipy.ndimage import interpolation

def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix

def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    
    return interpolation.affine_transform(image,affine,offset=offset)

In [4]:
#############-----ORDER-----#########
# This is function which gives the Clusters in the given shortest path order
def lms(order,lm):
    ordered_lm = np.zeros((num_clusters,2))
    j = 0
    for i in order:
	    ordered_lm[j,:] = lm[int(i),:]
	    j = j+1
    #print(ordered_lm)
    return(ordered_lm)

##################################################

In [5]:
num_clusters = 20

def newimg(img_no):
    x1,x2 = [],[]  # Declaring lists to get points    
    newim = deskew(train_images[img_no])  # Caling deskew function
    
    # Loop for getting coordinates from the image
    for i in range(0,28):
        for j in range(0,28):
            if(newim[i][j] >127):
                x1.append(i)
                x2.append(j)
    x= []
    x.append(x1)
    x.append(x2)
    #print(x1)
    #print(x2)  
    data= np.matrix(x)
    data= np.array(data)
    data= data.T

    #############      CLUSTERING       #################

    km = KMeans(n_clusters = num_clusters,n_init = 10,max_iter=300, tol = 1e-04,random_state = 0)
    km= km.fit(data)  # It fits the data given data and gives required points
    lm= km.cluster_centers_
    
    #print("\n---------------------Cluster centers---------------------\n")
    #print(lm)
    #print("\n---------------------------------------------------------\n")
    
    n= num_clusters
    lm2= []
    lm2= np.array(fun1(lm,n))  # Calling fun1 function above for getting the list
    print(lm2)
    list1= matrix2.fun3(lm2,n) # matrix2.fun3 gives the shortest path among all the points.
    list2=np.array(list1[0],dtype= np.int64)
    order= list2[:-1]
    mm= lms(order,lm)  # Calling lms function to get the ordered list
    return(mm)

In [6]:
def comp(n):
    lbl= []
    mm= []
    num= n
    
    for i in range(0,100):
        if(train_labels[i] == num):
            lbl.append(i)

    #print("Number: "+str(num)+"\n"+str(lbl))

    for i in range(0,len(lbl)):
        xnum= 13
        #l = list()
        #print("\n"+str(lbl[i]))
        try:
	        mm1= newimg(lbl[i])
            
        except:
            #continue
            print(lbl[i])
        #mm1= newimg(lbl[i])
        if(xnum < mm1[0,0]):
            #xnum= int(mm1[-1,-2]) - 5
            mm1= mm1[: :-1]
        mm.append(mm1.reshape(22,).T)
        
    return(mm)

listed= [] 
n=0
print("List of distances of the images\n")
#d= {i:comp(i) for i in range(0,10)}

List of distances of the images



In [7]:
# Storing all the distances in the one pickle file

import pickle as p
'''with open('c20.p','wb') as handle:
    p.dump(d,handle)''' 
with open('c20.p','rb') as handle:
    d= p.load(handle)

In [8]:
# Printing all the images shortest distances of the training distances
print(d)

{0: [array([ 8.875     , 14.125     ,  6.125     , 14.875     ,  6.        ,
       12.28571429,  8.16666667, 11.16666667, 10.        , 10.25      ,
       11.6       ,  8.8       , 13.8       ,  8.6       , 15.33333333,
        7.66666667, 18.        ,  8.5       , 20.83333333,  9.83333333,
       22.125     , 12.125     , 22.2       , 14.4       , 21.4       ,
       16.2       , 20.        , 18.        , 18.16666667, 19.83333333,
       15.71428571, 21.        , 13.75      , 21.        , 11.83333333,
       19.83333333,  9.2       , 18.6       ,  7.66666667, 17.33333333]), array([ 7.125     , 13.875     ,  4.83333333, 14.83333333,  6.5       ,
       17.        ,  8.4       , 18.2       , 10.71428571, 19.        ,
       13.5       , 20.5       , 16.8       , 21.6       , 18.2       ,
       20.4       , 20.5       , 18.5       , 21.91666667, 15.41666667,
       21.71428571, 12.14285714, 20.        ,  9.5       , 18.        ,
        8.        , 16.        ,  7.25      , 13.        

In [9]:
# This function uses the Clustering technique to cluster the all the images.
# After clustering, number representing clusters will be outputs of the function

def cl_im(i):
    km1 = KMeans(n_clusters = 25,n_init = 10,max_iter=300, tol = 1e-04,random_state = 0)
    km1= km1.fit(d[i])
    hollow1= km1.cluster_centers_
    return(hollow1)


dict_num= {n:cl_im(n) for n in range(0,10)}
print(dict_num)

{0: array([[16.13333333, 22.        , 13.16666667, 21.5       , 10.6       ,
        20.33333333,  7.93055556, 19.05555556,  8.3       , 16.98333333,
         6.19642857, 15.86309524,  6.0452381 , 13.81428571,  5.7962963 ,
        11.85185185,  7.33846154, 11.28498168,  9.43055556,  9.90277778,
        11.58253968,  8.97407407, 13.12289562,  7.51010101, 16.17857143,
         7.08333333, 19.2547619 ,  7.475     , 19.98677249,  9.12698413,
        21.97407407, 11.34444444, 21.8219697 , 14.18939394, 22.01984127,
        16.8452381 , 20.27777778, 18.79365079, 19.225     , 20.9452381 ],
       [ 4.        , 17.        ,  5.        , 14.5       ,  5.5       ,
        12.5       ,  7.        , 11.25      ,  9.8       , 10.6       ,
        12.        , 10.        , 14.5       , 10.        , 16.5       ,
        10.5       , 18.5       , 10.5       , 20.5       , 11.        ,
        20.66666667, 12.33333333, 21.75      , 14.        , 20.5       ,
        16.        , 18.5       , 17.5       ,

In [10]:
# Storing that number representing clusters into the pickle file

'''with open('c20d25.p','wb') as handle:
    p.dump(dict_num,handle)''' 
with open('c20d25.p','rb') as handle:
    dict_num= p.load(handle)
#print(dict_num[0][1])
print("\n\n-----Clusters are loaded-----\n\n")



-----Clusters are loaded-----


